# Trade Lots

In [1]:
from datetime import datetime
from urllib.parse import quote_plus, urlparse, parse_qs
from lxml import html, etree
import pandas as pd
import requests

url = "http://www.nasdaq.com/symbol/amd/time-sales?time=1&pageno={page}"
page = requests.get(url.format(page=1))
tree = html.fromstring(page.content)

pager = tree.find('.//ul[@id="pager"]')
page_links = pager.findall('.//a')
href = page_links[-1].attrib['href']
href

AttributeError: 'NoneType' object has no attribute 'findall'

In [8]:
parsed_url = urlparse(href)
query_params = parse_qs(parsed_url.query)
max_page = int(query_params['pageno'][0])
max_page

307

In [ ]:
def load_data(stocks, time, friday):
    
    num_worker_threads = 100
    
    def worker():
        while True:
            args = q.get()
            if args is None:
                break
            get_transactions(q, r, *args)  # <-- see definition of this function below
            q.task_done()

    q = queue.Queue() # queue with arguments for the 'get_transactions' function
    r = queue.Queue() # queue with raw responses

    threads = []
    for i in range(num_worker_threads):
        t = threading.Thread(target=worker)
        t.start()
        threads.append(t)

    for stock in stocks:
        q.put((stock, friday))

    # block until all tasks are done
    q.join()

    # stop workers
    for i in range(num_worker_threads):
        q.put(None)

    for t in threads:
        t.join()
        
    save_rows(r, time)

In [20]:
df = pd.DataFrame([], columns=('Time', 'Price', 'Volume'))
start_time = datetime.now()
i = 0
for page_num in range(1, max_page + 1):
    
    page = requests.get(url.format(page=page_num))
    tree = html.fromstring(page.content)
    
    table = tree.find('.//table[@id="AfterHoursPagingContents_Table"]')
    for row in table:
        if row.tag == 'tr':
            df.loc[i] = [col.text_content() for col in row]
            i += 1
            
print(datetime.now() - start)
df

0:04:13.965301


,Time,Price,Volume
0,09:59:59,$ 12.195,700
1,09:59:59,$ 12.20,700
2,09:59:59,$ 12.195,100
3,09:59:59,$ 12.20,100
4,09:59:59,$ 12.195,100
5,09:59:59,$ 12.19,279
6,09:59:58,$ 12.19,151
7,09:59:58,$ 12.1901,151
8,09:59:56,$ 12.197,"1,000"
9,09:59:55,$ 12.1918,500
